mcp/
 ├─ agents/
 │   ├─ forecast_agent.py
 │   ├─ reorder_agent.py
 │   ├─ routing_agent.py
 │   ├─ guardrails.py
 │
 ├─ data/
 │   ├─ inventory.xlsx
 │   ├─ sales_history.csv
 │
 ├─ graph/
 │   ├─ workflow.py
 │
 ├─ database/
 │   ├─ loader.py
 │   ├─ duckdb_handler.py (optional)
 │   ├─ sqlite_handler.py (optional)
 │
 ├─ llm/
 │   ├─ azure_llm.py
 │
 ├─ run.py
 ├─ requirements.txt
 ├─ .env


langgraph
langchain
langchain-openai
python-dotenv
pandas
openpyxl
pydantic


In [ ]:
import os
from langchain_openai import ChatOpenAI

def get_llm(model="azure/genailab-maas-gpt-4o-mini", temp=0):
    return ChatOpenAI(
        model=model,
        api_key=os.getenv("AZURE_MAAS_API_KEY"),
        base_url="https://models.inference.ai.azure.com",
        temperature=temp
    )


In [ ]:
import pandas as pd
import os

def load_inventory():
    file = "mcp/data/inventory.xlsx"
    df = pd.read_excel(file)
    df.columns = [c.lower() for c in df.columns]

    inv = {}
    for _, r in df.iterrows():
        inv[r["sku"]] = {
            "name": r["name"],
            "stock": int(r["stock"]),
            "threshold": int(r["threshold"])
        }
    return inv


def load_sales_history():
    file = "mcp/data/sales_history.csv"
    df = pd.read_csv(file)
    df.columns = [c.lower() for c in df.columns]
    return df


In [ ]:
from pydantic import BaseModel, Field, ValidationError

class UserQuery(BaseModel):
    query: str = Field(..., min_length=3)
    intent: str = Field(..., description="forecast | reorder | inventory")

FORBIDDEN = ["hack", "illegal", "explode", "virus"]

def validate_input(text: str):
    for bad in FORBIDDEN:
        if bad in text.lower():
            raise Exception("❌ Forbidden content detected.")
    return True


def parse_intent(user_query: str):
    q = user_query.lower()
    if "forecast" in q or "predict" in q:
        return "forecast"
    if "reorder" in q or "replenish" in q:
        return "reorder"
    if "stock" in q or "inventory" in q:
        return "inventory"
    return "inventory"


In [ ]:
from agents.guardrails import validate_input, parse_intent

def route_query(state):
    user_input = state["query"]
    validate_input(user_input)
    state["intent"] = parse_intent(user_input)
    return state


In [ ]:
from llm.azure_llm import get_llm
from database.loader import load_sales_history

llm = get_llm()

def forecasting_agent(state):
    sku = state.get("sku")
    sales = load_sales_history()

    if sku:
        sales = sales[sales["sku"] == sku]

    prompt = f"""
You are an enterprise forecasting agent.
Generate a demand forecast using:
- sales_velocity
- moving average trend
- seasonal indicators

Sales Data Sample:
{sales.head(10).to_string()}

Provide:
- 7-day forecast
- 30-day forecast
- Trend insights
"""

    resp = llm.invoke(prompt)
    state["output"] = resp.content
    return state


In [ ]:
from llm.azure_llm import get_llm
from database.loader import load_inventory

llm = get_llm()

def reorder_agent(state):
    sku = state.get("sku")
    inventory = load_inventory()

    if sku not in inventory:
        state["output"] = "SKU not found."
        return state

    item = inventory[sku]

    prompt = f"""
You are an enterprise supply chain replenishment agent.

Item:
SKU: {sku}
Name: {item['name']}
Current Stock: {item['stock']}
Threshold: {item['threshold']}

Tasks:
- Should we reorder?
- How much?
- Provide business justification.
- Add risk score.

Return JSON result.
"""

    resp = llm.invoke(prompt)
    state["output"] = resp.content
    return state


In [ ]:
from database.loader import load_inventory
from llm.azure_llm import get_llm

llm = get_llm()

def inventory_agent(state):
    sku = state.get("sku")
    inventory = load_inventory()

    if sku not in inventory:
        state["output"] = "SKU not found."
        return state

    data = inventory[sku]

    prompt = f"""
Provide an enterprise-grade inventory status summary.

Item:
SKU: {sku}
Name: {data['name']}
Stock: {data['stock']}
Threshold: {data['threshold']}

Tasks:
- Stock health
- Risk of stockout
- Next action
- Business notes
"""

    resp = llm.invoke(prompt)
    state["output"] = resp.content
    return state


In [ ]:
from langgraph.graph import StateGraph, END

from agents.routing_agent import route_query
from agents.forecast_agent import forecasting_agent
from agents.reorder_agent import reorder_agent
from agents.inventory_agent import inventory_agent

class AppState(dict):
    pass

graph = StateGraph(AppState)

graph.set_entry_point("route_query")

graph.add_node("route_query", route_query)
graph.add_node("forecast", forecasting_agent)
graph.add_node("reorder", reorder_agent)
graph.add_node("inventory", inventory_agent)

graph.add_conditional_edges(
    "route_query",
    lambda s: s["intent"],
    {
        "forecast": "forecast",
        "reorder": "reorder",
        "inventory": "inventory"
    }
)

graph.add_edge("forecast", END)
graph.add_edge("reorder", END)
graph.add_edge("inventory", END)

app = graph.compile()


In [ ]:
from graph.workflow import app

user_input = input("Enter request: ")  # e.g., "forecast SKU001"

# Smart SKU detection
sku = None
for word in user_input.split():
    if word.upper().startswith("SKU"):
        sku = word.upper()

result = app.invoke({
    "query": user_input,
    "sku": sku
})

print("\n====== SMART INVENTORY AGENT OUTPUT ======")
print(result["output"])
print("==========================================\n")
